In [1]:
import tweepy
import os
import glob
import time
import numpy as np
import pandas as pd
import re

In [35]:
s_round = '2021-22_regular'
division = 'v1_w'
path = '{0}/{1}/teamdatabase.csv'.format(s_round, division)
teamdatabase = pd.read_csv(path)

In [36]:
teamdatabase.columns

Index(['AA', 'AP', 'AE', 'ASucc%', 'AP/S', 'BAA', 'BAP', 'BAE', 'BASucc%',
       'BP', 'BP/S', 'SVA', 'SVP', 'SVE', 'SVx', 'SVEff%', 'RA', 'Rx', 'Rg',
       'RSucc%', 'MatchNo.', 'Date', 'Year', 'Month', 'Day', 'Place', 'Venue',
       'Team', 'Op.Team', 'Director', 'Coach', 'Win Set', 'Lose Set',
       'Total Set', 'Win Lose', 'Get Point', 'Lose Point', 'Game Time',
       'Start Time', 'End Time', 'Spectators', 'Jury', 'Chief Umpire',
       'Sub Umpire', 'Judgeman', 'Home Away', 'OpAA', 'OpAP', 'OpAE',
       'OpASucc%', 'OpAP/S', 'OpBAA', 'OpBAP', 'OpBAE', 'OpBASucc%', 'OpBP',
       'OpBP/S', 'OpSVA', 'OpSVP', 'OpSVE', 'OpSVx', 'OpSVEff%', 'OpRA',
       'OpRx', 'OpRg', 'OpRSucc%'],
      dtype='object')

In [41]:
win_data = teamdatabase[['Date','Team','Win Lose']].copy()
# win_data[win_data['Win Lose']=='Win']
f1 = lambda x: 1 if x == 'Win' else 0
f2 = lambda x: 1 if x == 'Lose' else 0
win_data['Win'] = win_data['Win Lose'].apply(f1)
win_data['Lose'] = win_data['Win Lose'].apply(f2)
# win_data['Game'] = win_data['Win'] + win_data['Lose']

# 日付設定
# date_index = pd.date_range(start='2021-10-14', end='2022-03-15').to_series().dt.strftime("%Y/%m/%d")
# time_tables = pd.DataFrame(date_index, columns=['Date'])


win_data.drop('Win Lose', axis=1, inplace=True)
teams = list(win_data.drop_duplicates(keep='first', subset=['Team'])['Team'].values)
for i, team in enumerate(teams):
#     print(i, team)
    win_team_data = win_data[win_data['Team']==team].drop('Team', axis=1)
    win_team_data[team] = win_team_data['Win'].cumsum()
    
#     win_team_data[['Win', 'Lose']] = win_team_data[['Win', 'Lose']].cumsum()
#     win_team_data['Game'] = win_team_data['Win'] + win_team_data['Lose']
#     win_team_data[team] = (win_team_data['Win']*100 / win_team_data['Game']).round(1)
    time_table = win_team_data[['Date', team]]
#     print(time_table.head(2))
    if i == 0:
        time_tables = time_table
        continue
    time_tables = pd.merge(time_tables, time_table, how='outer', on='Date')


time_tables = pd.concat([time_tables, time_tables])
# time_start_tables = pd.DataFrame(data='開幕前', index=['開幕前'], columns=['Date', team])
time_tables = time_tables.sort_values('Date')

time_start_tables = pd.DataFrame(data=0.0, index=['開幕前'], columns=time_tables.columns)
time_start_tables['Date'] = '開幕前'
time_start_tables = pd.concat([time_start_tables, time_start_tables])

time_tables = time_tables.interpolate('ffill').fillna(0)
time_tables['Date'] = time_tables['Date'].str.replace('-', '/')
time_tables = pd.concat([time_start_tables, time_tables])
time_tables.tail()

,Date,東レアローズ,ヴィクトリーナ姫路,トヨタ車体クインシーズ,JTマーヴェラス,NECレッドロケッツ,KUROBEアクアフェアリーズ,デンソーエアリービーズ,PFUブルーキャッツ,久光スプリングス,日立Astemoリヴァーレ,岡山シーガルズ,埼玉上尾メディックス
30,2022/03/30,24.0,8.0,10.0,24.0,23.0,3.0,18.0,12.0,21.0,13.0,11.0,18.0
37,2022/04/02,24.0,8.0,10.0,25.0,23.0,3.0,19.0,12.0,22.0,14.0,11.0,19.0
37,2022/04/02,24.0,8.0,10.0,25.0,23.0,3.0,19.0,12.0,22.0,14.0,11.0,19.0
31,2022/04/03,25.0,8.0,10.0,26.0,23.0,3.0,20.0,12.0,23.0,14.0,11.0,20.0
31,2022/04/03,25.0,8.0,10.0,26.0,23.0,3.0,20.0,12.0,23.0,14.0,11.0,20.0


In [42]:
# みなし試合考慮
for i in range(2):
    time_tables.loc['v1_w{}'.format(i)] = ['現在(みなし試合含む)',26,8,10,27,23,3,20,12,23,14,12,20]
#     time_tables.loc['v2_w{}'.format(i)] = ['最終結果(みなし試合含む)',7,5,13,8,16,7,14,8,2,5]
#     time_tables.loc['v1_m{}'.format(i)] = ['現在(みなし試合含む)',15,15,5,25,24,6,27,9,25,27]
#     time_tables.loc['v2_m{}'.format(i)] = ['最終結果(みなし試合含む)',10,3,11,10,27,20,15,24,13,13,25,7,12,12,8]
#     time_tables.loc['v3_m{}'.format(i)] = ['最終結果(みなし試合含む)',7,13,1,9]
time_tables.set_index('Date', inplace=True)
time_tables

,東レアローズ,ヴィクトリーナ姫路,トヨタ車体クインシーズ,JTマーヴェラス,NECレッドロケッツ,KUROBEアクアフェアリーズ,デンソーエアリービーズ,PFUブルーキャッツ,久光スプリングス,日立Astemoリヴァーレ,岡山シーガルズ,埼玉上尾メディックス
Date,,,,,,,,,,,,
開幕前,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
開幕前,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021/10/15,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021/10/15,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021/10/16,2.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022/04/02,24.0,8.0,10.0,25.0,23.0,3.0,19.0,12.0,22.0,14.0,11.0,19.0
2022/04/03,25.0,8.0,10.0,26.0,23.0,3.0,20.0,12.0,23.0,14.0,11.0,20.0
2022/04/03,25.0,8.0,10.0,26.0,23.0,3.0,20.0,12.0,23.0,14.0,11.0,20.0


In [39]:
dict_att_color = {
    'v1_m':{
        'ジェイテクトSTINGS': '#b2b2b2',
        'JTサンダーズ広島': '#00cb65',
        'VC長野トライデンツ': '#980000',
        '東レアローズ': '#0000ff',
        '堺ブレイザーズ': '#ffff00',
        '大分三好ヴァイセアドラー': '#3232ff',
        'サントリーサンバーズ': '#ff003f',
        'FC東京': '#0032cb',
        'パナソニックパンサーズ': '#000065',
        'ウルフドッグス名古屋': '#000000'
    },
    'v2_m':{
        '警視庁フォートファイターズ': '#b2b2b2',
        '千葉ZELVA': '#9932ff',
        'トヨタ自動車サンホークス': '#004c98',
        '奈良ドリーマーズ': '#006532',
        'ヴォレアス北海道': '#cb0032',
        '埼玉アザレア': '#cb0098',
        'サフィルヴァ北海道': '#ffff00',
        'ヴィアティン三重': '#ff3f00',
        '大同特殊鋼レッドスター': '#984c00',
        '兵庫デルフィーノ': '#007fff',
        '富士通カワサキレッドスピリッツ': '#ff0000',
        'クボタスピアーズ': '#00ffff',
        'つくばユナイテッドSunGAIA': '#009872',
        'きんでんトリニティーブリッツ': '#00984c',
        '東京ヴェルディ': '#009826',        
    },
    'v3_m':{
        '長野GaRons': '#003fff',
        'アイシンティルマーレ': '#3299ff',
        'トヨタモビリティ東京スパークル': '#ff3232',
        '近畿クラブスフィーダ': '#000000'
    },
    'v1_w':{
        '東レアローズ':'#0018fa', 
        'ヴィクトリーナ姫路':'#fa00bd',
        'トヨタ車体クインシーズ':'#0a1899',
        'JTマーヴェラス':'#00b348',
        'NECレッドロケッツ': '#ff003f',
        'KUROBEアクアフェアリーズ': '#00ebe3',
        'デンソーエアリービーズ': '#ff3232',
        'PFUブルーキャッツ': '#1000fa',
        '久光スプリングス': '#7ce01f',
        '日立Astemoリヴァーレ': '#8b00bb',
        '岡山シーガルズ': '#8adeff',
        '埼玉上尾メディックス': '#000000'
    },
    'v2_w':{
        'ブレス浜松': '#32ccff',
        'JAぎふリオレーナ': '#99e5ff',
        'プレステージ・インターナショナルアランマーレ': '#ff3f00',
        '千葉エンゼルクロス': '#980026',
        '群馬銀行グリーンウイングス': '#00cb65',
        'リガーレ仙台': '#ff0000',
        'ルートインホテルズブリリアントアリーズ': '#ff99e5',
        'GSS東京サンビームズ': '#269800',
        'フォレストリーヴズ熊本': '#00984c',
        '大野石油広島オイラーズ': '#3265ff'
    }
}

In [43]:
import bar_chart_race as bcr
import japanize_matplotlib
# bcr.bar_chart_race(df = time_tables)
# 描画
bcr.bar_chart_race(df=time_tables, 
                   cmap=[dict_att_color[division][x] for x in dict_att_color[division].keys()])

In [13]:
date_index = pd.date_range(start='2021-10-14', end='2022-03-15').to_series().dt.strftime("%Y/%m/%d")
date_index

2021-10-14    2021/10/14
2021-10-15    2021/10/15
2021-10-16    2021/10/16
2021-10-17    2021/10/17
2021-10-18    2021/10/18
                 ...    
2022-03-11    2022/03/11
2022-03-12    2022/03/12
2022-03-13    2022/03/13
2022-03-14    2022/03/14
2022-03-15    2022/03/15
Freq: D, Length: 153, dtype: object

In [8]:
da_teams = teams.copy()
da_teams.append('Date')
print(da_teams)

['ジェイテクトSTINGS', 'JTサンダーズ広島', 'VC長野トライデンツ', '東レアローズ', '堺ブレイザーズ', '大分三好ヴァイセアドラー', 'サントリーサンバーズ', 'FC東京', 'パナソニックパンサーズ', 'ウルフドッグス名古屋', 'Date']


In [50]:
teams

['東レアローズ',
 'ヴィクトリーナ姫路',
 'トヨタ車体クインシーズ',
 'JTマーヴェラス',
 'NECレッドロケッツ',
 'KUROBEアクアフェアリーズ',
 'デンソーエアリービーズ',
 'PFUブルーキャッツ',
 '久光スプリングス',
 '日立Astemoリヴァーレ',
 '岡山シーガルズ',
 '埼玉上尾メディックス']

In [10]:
dict_att_color[division].keys()

dict_keys(['ジェイテクトSTINGS', 'JTサンダーズ広島', 'VC長野トライデンツ', '東レアローズ', '堺ブレイザーズ', '大分三好ヴァイセアドラー', 'サントリーサンバーズ', 'FC東京', 'パナソニックパンサーズ', 'ウルフドッグス名古屋'])

In [44]:
s_round = '2021-22_regular'
division = 'v1_w'
path = '{0}/{1}/playerdatabase.csv'.format(s_round, division)
playerdatabase = pd.read_csv(path)

In [45]:
playerdatabase.columns

Index(['No.', 'L', 'Player', 'Set', '1', '2', '3', '4', '5', 'AA', 'AP', 'AE',
       'ASucc%', 'AP/S', 'BAA', 'BAP', 'BAE', 'BASucc%', 'BP', 'BP/S', 'SVA',
       'SVP', 'SVE', 'SVx', 'SVEff%', 'RA', 'Rx', 'Rg', 'RSucc%', 'MatchNo.',
       'Date', 'Year', 'Month', 'Day', 'Place', 'Venue', 'Team', 'Op.Team',
       'Director', 'Coach', 'Win Set', 'Lose Set', 'Total Set', 'Win Lose',
       'Get Point', 'Lose Point', 'Game Time', 'Start Time', 'End Time',
       'Spectators', 'Jury', 'Chief Umpire', 'Sub Umpire', 'Judgeman',
       'Home Away'],
      dtype='object')

In [46]:
player_stats = playerdatabase[['Date', 'Player', 'AP', 'BP', 'SVP']].copy()
player_stats['Total'] = player_stats['AP'] + player_stats['BP'] + player_stats['SVP']
total = player_stats[['Date', 'Player', 'Total']]
total

,Date,Player,Total
0,2021-10-15,井上奈々朱,6
1,2021-10-15,白井美沙紀,0
2,2021-10-15,中田紫乃,0
3,2021-10-15,中島未来,0
4,2021-10-15,クランヤナ,27
...,...,...,...
5444,2022-04-03,目黒優佳,0
5445,2022-04-03,西川有喜,2
5446,2022-04-03,ヒックマンジャスティス,0
5447,2022-04-03,小幡真子,0


In [47]:
# win_data.drop('Win Lose', axis=1, inplace=True)
players = list(total.drop_duplicates(keep='first', subset=['Player'])['Player'].values)
for i, player in enumerate(players):
#     print(i, team)
    total_data = total[total['Player']==player].drop('Player', axis=1)
    total_data[player] = total_data['Total'].cumsum()
    
#     win_team_data[['Win', 'Lose']] = win_team_data[['Win', 'Lose']].cumsum()
#     win_team_data['Game'] = win_team_data['Win'] + win_team_data['Lose']
#     win_team_data[team] = (win_team_data['Win']*100 / win_team_data['Game']).round(1)
    time_table = total_data[['Date', player]]
#     print(time_table.head(2))
    if i == 0:
        time_tables = time_table
        continue
    time_tables = pd.merge(time_tables, time_table, how='outer', on='Date')


time_tables = pd.concat([time_tables, time_tables])
# time_start_tables = pd.DataFrame(data='開幕前', index=['開幕前'], columns=['Date', team])
time_tables = time_tables.sort_values('Date')

time_start_tables = pd.DataFrame(data=0.0, index=['開幕前'], columns=time_tables.columns)
time_start_tables['Date'] = '開幕前'
time_start_tables = pd.concat([time_start_tables, time_start_tables])

time_tables = time_tables.interpolate('ffill').fillna(0)
time_tables['Date'] = time_tables['Date'].str.replace('-', '/')
time_tables = pd.concat([time_start_tables, time_tables])
time_tables.set_index('Date', inplace=True)
time_tables

,井上奈々朱,白井美沙紀,中田紫乃,中島未来,クランヤナ,小川愛里奈,関菜々巳,水杉玲奈,石川真佑,野呂加南子,...,間橋香織,佐藤彩乃,中山夏菜子,岩崎こよみ,高間来瞳,吉田美海,堤亜里菜,深澤つぐみ,上野香織,古川あおいアン
Date,,,,,,,,,,,,,,,,,,,,,
開幕前,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
開幕前,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021/10/15,6.0,0.0,0.0,0.0,27.0,17.0,2.0,0.0,15.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021/10/15,6.0,0.0,0.0,0.0,27.0,17.0,2.0,0.0,15.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021/10/16,6.0,0.0,0.0,0.0,46.0,27.0,5.0,0.0,31.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022/03/30,138.0,5.0,119.0,0.0,776.0,288.0,57.0,0.0,487.0,51.0,...,0.0,0.0,27.0,12.0,2.0,0.0,1.0,8.0,0.0,3.0
2022/04/02,138.0,5.0,119.0,0.0,776.0,288.0,57.0,0.0,487.0,51.0,...,0.0,0.0,27.0,18.0,2.0,0.0,1.0,8.0,0.0,3.0
2022/04/02,138.0,5.0,119.0,0.0,776.0,288.0,57.0,0.0,487.0,51.0,...,0.0,0.0,27.0,18.0,2.0,0.0,1.0,8.0,0.0,3.0


In [48]:
# players = list(total.drop_duplicates(keep='first', subset=['Player'])['Player'].values)
player_team = playerdatabase[['Player', 'Team']].drop_duplicates(keep='first', subset=['Player'])
# player_team
hex_colors = dict()
for i in player_team.values:
#     print(i[0], i[1])
    hex_colors[i[0]] = dict_att_color[division][i[1]]
# hex_colors

In [49]:
import warnings
warnings.simplefilter('ignore')
bcr.bar_chart_race(df=time_tables,
                   n_bars=10, 
                   cmap=[hex_colors[x] for x in hex_colors.keys()])